Question : What is the predicted energy output of the solar panels over the next week/month/year?

<a id="1"></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#2b2924;font-family:newtimeroman;color:#cc7e08;font-size:160%;text-align:center;border-radius:10px 10px;">Dataset</p> 
    
    
More information about the dataset you can find [here](https://www.kaggle.com/datasets/anikannal/solar-power-generation-data?datasetId=836676&sortBy=voteCount).

<a id="2"></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#2b2924;font-family:newtimeroman;color:#cc7e08;font-size:160%;text-align:center;border-radius:10px 10px;">Libraries</p> 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

<a id="3"></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#2b2924;font-family:newtimeroman;color:#cc7e08;font-size:160%;text-align:center;border-radius:10px 10px;">Data Loading</p> 

In [2]:
# power generation datasets
g1 = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv")
g1.head()
g2 = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_2_Generation_Data.csv")
g2.head()


,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15 00:00:00,4136001,4UPUqMRk7TRMgml,0.0,0.0,9425.000000,2.429011e+06
1,2020-05-15 00:00:00,4136001,81aHJ1q11NBPMrL,0.0,0.0,0.000000,1.215279e+09
2,2020-05-15 00:00:00,4136001,9kRcWv60rDACzjR,0.0,0.0,3075.333333,2.247720e+09
3,2020-05-15 00:00:00,4136001,Et9kgGMDl729KT4,0.0,0.0,269.933333,1.704250e+06
4,2020-05-15 00:00:00,4136001,IQ2d7wF4YD8zU1Q,0.0,0.0,3177.000000,1.994153e+07


In [3]:
# sensor datasets
s1 = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv")
s1.head()
s2 = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv")
s2.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4136001,iq8k7ZNt4Mwm3w0,27.004764,25.060789,0.0
1,2020-05-15 00:15:00,4136001,iq8k7ZNt4Mwm3w0,26.880811,24.421869,0.0
2,2020-05-15 00:30:00,4136001,iq8k7ZNt4Mwm3w0,26.682055,24.427290,0.0
3,2020-05-15 00:45:00,4136001,iq8k7ZNt4Mwm3w0,26.500589,24.420678,0.0
4,2020-05-15 01:00:00,4136001,iq8k7ZNt4Mwm3w0,26.596148,25.088210,0.0


In [4]:
# power generation datasets
def check(data):
    l = []
    columns = data.columns
    for col in columns:
        instances = data[col].count()
        dtypes = data[col].dtype
        unique = data[col].nunique()
        sum_null = data[col].isnull().sum()
        duplicates = data.duplicated().sum()
        l.append([col, dtypes, instances, unique, sum_null, duplicates])
    data_check = pd.DataFrame(l, columns=["column", "dtype", "instances", "unique", "sum_null", "duplicates"])
    return data_check

check1 = check(g1)
check2 = check(g2)

output = pd.concat([check1, check2], axis=1)

output

# g1.info()
# print("\n")
# g1.describe()

# g2.info()
# print("\n")
# g2.describe()

,column,dtype,instances,unique,sum_null,duplicates,column,dtype,instances,unique,sum_null,duplicates
0,DATE_TIME,object,68778,3158,0,0,DATE_TIME,object,67698,3259,0,0
1,PLANT_ID,int64,68778,1,0,0,PLANT_ID,int64,67698,1,0,0
2,SOURCE_KEY,object,68778,22,0,0,SOURCE_KEY,object,67698,22,0,0
3,DC_POWER,float64,68778,32909,0,0,DC_POWER,float64,67698,30825,0,0
4,AC_POWER,float64,68778,32686,0,0,AC_POWER,float64,67698,30783,0,0
5,DAILY_YIELD,float64,68778,29900,0,0,DAILY_YIELD,float64,67698,30490,0,0
6,TOTAL_YIELD,float64,68778,37267,0,0,TOTAL_YIELD,float64,67698,33115,0,0


<div style="border-radius: 40px;
            color: #403b34;
            background-color: #bdbebf;
            font-size: 100%;
            text-align: left;
            padding: 20px;
            margin: 10px;
            box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.1);">
    <p style="margin-bottom: 10px; font-weight: bold;">Output Analysis:</p>
    <ul style="margin-top: 0;">
        <li>power generation dataset of Plant 1 comprises a total of 68,778 records</li>
        <li>power generation dataset of Plant 2 comprises a total of 67,698 records
        <li>there are neither duplicates nor any missing values in both power generation datasets</li>
        <li>each of the two plants is equipped with a total of 22 inverters</li>
        <li>'DATE_TIME' column is currently stored as an object data type instead of date-time in both power generation datasets</li>
    </ul>
</div>

In [5]:
# sensor datasets
def check(data):
    l = []
    columns = data.columns
    for col in columns:
        instances = data[col].count()
        dtypes = data[col].dtype
        unique = data[col].nunique()
        sum_null = data[col].isnull().sum()
        duplicates = data.duplicated().sum()
        l.append([col, dtypes, instances, unique, sum_null, duplicates])
    data_check = pd.DataFrame(l, columns=["column", "dtype", "instances", "unique", "sum_null", "duplicates"])
    return data_check

check1 = check(s1)
check2 = check(s2)

output = pd.concat([check1, check2], axis=1)

output

# s1.info()
# print("\n")
# s1.describe()

# s2.info()
# print("\n")
# s2.describe()

,column,dtype,instances,unique,sum_null,duplicates,column,dtype,instances,unique,sum_null,duplicates
0,DATE_TIME,object,3182,3182,0,0,DATE_TIME,object,3259,3259,0,0
1,PLANT_ID,int64,3182,1,0,0,PLANT_ID,int64,3259,1,0,0
2,SOURCE_KEY,object,3182,1,0,0,SOURCE_KEY,object,3259,1,0,0
3,AMBIENT_TEMPERATURE,float64,3182,3182,0,0,AMBIENT_TEMPERATURE,float64,3259,3259,0,0
4,MODULE_TEMPERATURE,float64,3182,3182,0,0,MODULE_TEMPERATURE,float64,3259,3259,0,0
5,IRRADIATION,float64,3182,1758,0,0,IRRADIATION,float64,3259,1863,0,0


<div style="border-radius: 40px;
            color: #403b34;
            background-color: #bdbebf;
            font-size: 100%;
            text-align: left;
            padding: 20px;
            margin: 10px;
            box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.1);">
    <p style="margin-bottom: 10px; font-weight: bold;">Output Analysis:</p>
    <ul style="margin-top: 0;">
        <li>sensor dataset of Plant 1 comprises a total of 3,182 records</li>
        <li>sensor generation dataset of Plant 2 comprises a total of 3,259 records
        <li>there are neither duplicates nor any missing values in both  sensor datasets</li>
        <li>'DATE_TIME' column is currently stored as an object data type instead of date-time in both sensor datasets</li>
    </ul>
</div>